In [1]:
import numpy as np

from Utils import Utils
utils = Utils()
findings = utils.findings
#findings.loc[(findings['finding_categories'] == "['Mass']") & (findings['breast_birads'] == 'BI-RADS 5') & (findings['view_position'] == 'CC') & (findings['laterality'] == 'R')]

In [ ]:
# 66d5d94402456f664753ec443d077bf9
image_id = '66d5d94402456f664753ec443d077bf9'
image_path = utils.get_path(image_id)
dicom = pydicom.dcmread(image_path)
img = dicom.pixel_array

# Extraer Window Center y Window Width
WC = dicom.WindowCenter
WW = dicom.WindowWidth

print(f"Window center: {WC}, Window width: {WW}")

# Aplicar la operación de windowing
img_windowed = np.clip((img - (WC - WW / 2)) / WW * 255.0, 0, 255)

#img_windowed = apply_voi_lut(img, dicom)

# Convertir la imagen a uint8
img_windowed = img_windowed.astype(np.uint8)

# Coordenadas de la bounding box (en este caso, de ejemplo)
# Vértice superior izquierdo
x1, y1 = int(ss1.loc[ss1['image_id'] == image_id, 'xmin'].values[0]), int(ss1.loc[ss1['image_id'] == image_id, 'ymin'].values[0])
# Vértice inferior derecho
x2, y2 = int(ss1.loc[ss1['image_id'] == image_id, 'xmax'].values[0]), int(ss1.loc[ss1['image_id'] == image_id, 'ymax'].values[0])

# Dibujar la bounding box en la imagen
# cv2.rectangle(image, start_point, end_point, color, thickness)
#cv2.rectangle(img_windowed, (x1, y1), (x2, y2), (255, 0, 0), 2)  # Color azul en BGR, grosor 2


ymin = 0   # Mínimo valor en la escala de salida
ymax = 255 # Máximo valor en la escala de salida

# Crear una copia para no modificar la imagen original
img_windowed2 = np.zeros_like(img, dtype=np.float32)

for i in range(img.shape[0]):
    for j in range(img.shape[1]):
        x = img[i, j]
        if x <= WC - 0.5 - (WW - 1) / 2:
            img_windowed2[i, j] = ymin
        elif x > WC - 0.5 + (WW - 1) / 2:
            img_windowed2[i, j] = ymax
        else:
            img_windowed2[i, j] = ((x - (WC - 0.5)) / (WW - 1) + 0.5) * (ymax - ymin) + ymin

# Convertir la imagen a uint8 para su visualización
img_windowed2 = img_windowed2.astype(np.uint8)

# Concatenar las imágenes horizontalmente
combined_image = np.hstack((img_windowed, img_windowed2))

# Mostrar la imagen combinada
cv2.imshow('Combined Image', combined_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [9]:
utils.findings

,study_id,series_id,image_id,laterality,view_position,height,width,breast_birads,breast_density,finding_categories,finding_birads,xmin,ymin,xmax,ymax,split
0,48575a27b7c992427041a82fa750d3fa,26de4993fa6b8ae50a91c8baf49b92b0,4e3a578fe535ea4f5258d3f7f4419db8,R,CC,3518,2800,BI-RADS 4,DENSITY C,['Mass'],BI-RADS 4,2355.139893,1731.640015,2482.979980,1852.750000,training
1,48575a27b7c992427041a82fa750d3fa,26de4993fa6b8ae50a91c8baf49b92b0,dac39351b0f3a8c670b7f8dc88029364,R,MLO,3518,2800,BI-RADS 4,DENSITY C,['Mass'],BI-RADS 4,2386.679932,1240.609985,2501.800049,1354.040039,training
2,75e8e48933289d70b407379a564f8594,853b70e7e6f39133497909d9ca4c756d,c83f780904f25eacb44e9030f32c66e1,R,CC,3518,2800,BI-RADS 3,DENSITY C,['Global Asymmetry'],BI-RADS 3,2279.179932,1166.510010,2704.439941,2184.260010,training
3,75e8e48933289d70b407379a564f8594,853b70e7e6f39133497909d9ca4c756d,893528bc38a0362928a89364f1b692fd,R,MLO,3518,2800,BI-RADS 3,DENSITY C,['Global Asymmetry'],BI-RADS 3,1954.270020,1443.640015,2589.760010,2193.810059,training
4,c3487424fee1bdd4515b72dc3fd69813,77619c914263eae44e9099f1ce07192c,318264c881bf12f2c1efe5f93920cc37,R,CC,3518,2800,BI-RADS 4,DENSITY C,['Architectural Distortion'],BI-RADS 4,2172.300049,1967.410034,2388.699951,2147.159912,training
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20481,f2093a752e6b44df5990f5fd38c99dd2,2b1b2b8f48abab9819c0b3d091e152ee,ea732154d149f619b20070b78060ae65,R,CC,2812,2012,BI-RADS 2,DENSITY C,['No Finding'],NaN,NaN,NaN,NaN,NaN,training
20482,b3c8969cd2accfa4dbb2aece1f7158ab,69d7f07ea04572dad5e5aa62fbcfc4b7,4689616c3d0b46fcba7a771107730791,R,CC,3580,2702,BI-RADS 2,DENSITY C,['No Finding'],NaN,NaN,NaN,NaN,NaN,training
20483,b3c8969cd2accfa4dbb2aece1f7158ab,69d7f07ea04572dad5e5aa62fbcfc4b7,3c22491bcf1d0b004715c28d80981cdd,L,CC,3580,2702,BI-RADS 2,DENSITY C,['No Finding'],NaN,NaN,NaN,NaN,NaN,training
20484,b3c8969cd2accfa4dbb2aece1f7158ab,69d7f07ea04572dad5e5aa62fbcfc4b7,d443b9725e331b8b27589aa725597801,R,MLO,3580,2686,BI-RADS 2,DENSITY C,['No Finding'],NaN,NaN,NaN,NaN,NaN,training
